In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Import data
incomeOpenData = pq.read_table('./distributedFolder/incomeOpenData/')
squareMeterPriceEvolution = pq.read_table('./distributedFolder/squareMeterPriceEvolution/')

incomeOpenData = incomeOpenData.to_pandas()
squareMeterPriceEvolution = squareMeterPriceEvolution.to_pandas()

In [4]:
# Join Tables and choose the useful columns
merged_df = incomeOpenData[['district_id','neighborhood_id','district','neighborhood','year','pop','RFD']].merge(squareMeterPriceEvolution[['district','neighborhood','year','preu_m2']], on=['district', 'year','neighborhood'])
merged_df

,district_id,neighborhood_id,district,neighborhood,year,pop,RFD,preu_m2
0,Q64124,Q1026658,Eixample,La Nova Esquerra de l'Eixample,2011,57659,107.7,2.314
1,Q64124,Q1026658,Eixample,La Nova Esquerra de l'Eixample,2009,58559,114.3,2.973
2,Q64124,Q1026658,Eixample,La Nova Esquerra de l'Eixample,2010,58247,115.2,2.802
3,Q64124,Q1026658,Eixample,La Nova Esquerra de l'Eixample,2008,58412,116.6,3.337
4,Q250935,Q1167101,Sant Martí,La Vila Olímpica del Poblenou,2011,9191,146.0,2.377
...,...,...,...,...,...,...,...,...
287,Q753075,Q980253,Sants-Montjuïc,"""El Poble-sec, Barcelona""",2008,40381,70.8,2.817
288,Q1650230,Q990510,Sant Andreu,Sant Andreu de Palomar,2011,55902,78.0,1.684
289,Q1650230,Q990510,Sant Andreu,Sant Andreu de Palomar,2009,55813,82.1,2.058
290,Q1650230,Q990510,Sant Andreu,Sant Andreu de Palomar,2010,55813,78.6,1.949


In [5]:
# Clean data type and format
merged_df['preu_m2'].replace('null', np.nan, inplace=True)
merged_df['preu_m2'] = merged_df['preu_m2'].astype(float)

In [6]:
# Missing values imputation
if merged_df['RFD'].isnull().values.any() == True:
    merged_df['RFD'].fillna(merged_df['RFD'].mean(), inplace=True)

if merged_df['preu_m2'].isnull().values.any() == True:
    merged_df['preu_m2'].fillna(merged_df['preu_m2'].mean(), inplace=True)

In [11]:
# Split the dataframes to apply different transformations
model_df = merged_df[['district','year','RFD','pop','preu_m2']]
dashboards_df = merged_df[['district_id','neighborhood_id','district','neighborhood','year','RFD','preu_m2']]

In [8]:
# Normalization
scaler = MinMaxScaler()
model_df['RFD'] = scaler.fit_transform(model_df[['RFD']])
model_df['preu_m2'] = scaler.fit_transform(model_df[['preu_m2']])

C:\Users\jordi\AppData\Local\Temp\ipykernel_2184\164989798.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['RFD'] = scaler.fit_transform(model_df[['RFD']])
C:\Users\jordi\AppData\Local\Temp\ipykernel_2184\164989798.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['preu_m2'] = scaler.fit_transform(model_df[['preu_m2']])


In [9]:
# One hot encoding for model dataframe
model_df = pd.get_dummies(model_df, columns=['district'])


In [12]:
# Save the dashboard dataset in a csv file
dashboards_df.to_csv('./ExploitationZone/Dashboard/dashboard.csv', index=False)

# Save the model dataset in a csv file
model_df.to_csv('./ExploitationZone/Model/model.csv', index=False)